# ICAPS24 SkDecide Tutorial: solving PDDL problems with classical planning, and reinforcement learning solvers

This notebook will show how to solve PDDL problems in scikit-decide via the great [Unified Planning](https://unified-planning.readthedocs.io/en/latest/) framework and its third-party engines from the [AIPlan4EU](https://github.com/aiplan4eu) project. We will also demonstrate how to call scikit-decide solvers from Unified Planning, allowing for solving PDDL problems with simulation-based solvers embedded in scikit-decide.

## Environment setup (package installation)

First we install scikit-decide.

<div class="alert alert-block alert-warning"><b>Warning: </b> This notebook currently needs the nightly version of scikit-decide in which a bug required to solve problems in this notebook has been corrected since the v1.0.0 release (the last scikit-decide release at the time we are writing this notebook). </div>

In [ ]:
!wget https://raw.githubusercontent.com/fteicht/icaps24-skdecide-tutorial/main/notebooks/install_skdecide.py

from install_skdecide import install_skdecide
install_skdecide(using_nightly_version=True, force_reinstall=True)

Second, we import the packages that will be used in this notebook.

In [24]:
import os
import sys

import networkx as nx
from enum import Enum
import datetime
import folium

import unified_planning as up
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import (
    OneshotPlanner,
    UserType,
    RealType,
    IntType,
    BoolType,
    Fluent,
    Problem,
    InstantaneousAction,
    SimulatedEffect,
    Equals,
    GE,
    Not,
    Or,
    Int,
    Object
)
from unified_planning.environment import get_environment

from skdecide.hub.domain.up import UPDomain
from skdecide.hub.solver.up import UPSolver
from skdecide.utils import rollout

from skdecide.hub.solver.iw import IW
from skdecide.hub.solver.ray_rllib import RayRLlib
from ray.rllib.algorithms.dqn import DQN

from skdecide.hub.domain.flight_planning.domain import (
    FlightPlanningDomain,
    WeatherDate
)

## Solving PDDL problems via the scikit-decide bridge to Unified Planning solvers

For the purpose of demonstration, we show how to solve a simplistic `blocksworld` instance with 4 blocks. Since we are relying on PDDL engines from Unified Planning (e.g. `fast-downward`, `ENHSP`, `tamer`, etc.), you are free to try more challenging benchmarks!

In [2]:
if not os.path.exists('bw-domain.pddl'):
    !wget https://raw.githubusercontent.com/potassco/pddl-instances/master/ipc-2000/domains/blocks-strips-typed/domain.pddl
    !mv domain.pddl bw-domain.pddl

if not os.path.exists('bw-instance.pddl'):
    !wget https://raw.githubusercontent.com/potassco/pddl-instances/master/ipc-2000/domains/blocks-strips-typed/instances/instance-1.pddl
    !mv instance-1.pddl bw-instance.pddl

reader = PDDLReader()
up_problem = reader.parse_problem('bw-domain.pddl', 'bw-instance.pddl')
up_problem.add_quality_metric(
    up.model.metrics.MinimizeSequentialPlanLength()
)

We now create a `skdecide.hub.domain.UPDomain` which embeds a Unified Planning [problem](https://unified-planning.readthedocs.io/en/latest/problem_representation.html#).

In [3]:
domain_factory = lambda: UPDomain(up_problem)
domain = domain_factory()

Once the `UPDomain` is created, we can call the `skdecide.hub.solver.UPSolver` which forward the solving process to a Unified Planning engine, then re-casting back the plan into the scikit-decide action format as defined in the `skdecide.hub.domain.UPDomain`.

We are specifically calling here the `fast-downward` [engine](https://github.com/aiplan4eu/up-fast-downward), after what we execute the resulting plan by using `skdecide.utils.rollout()`.

In [4]:
assert UPSolver.check_domain(domain)
with UPSolver(
    domain_factory=domain_factory,
    operation_mode=OneshotPlanner,
    name="fast-downward",
    engine_params={"output_stream": sys.stdout},
) as solver:
    solver.solve()
    rollout(
        domain,
        solver,
        num_episodes=1,
        max_steps=100,
        max_framerate=30,
        outcome_formatter=None,
    )

2024-05-31 10:51:35,897 | skdecide.utils | DEBUG | Logger is in verbose mode: all debug messages will be there for you to enjoy （〜^∇^ )〜
2024-05-31 10:51:35,898 | skdecide.utils | DEBUG | Episode 1 started with following observation:
2024-05-31 10:51:35,898 | skdecide.utils | DEBUG | {clear(c): true, clear(a): true, clear(b): true, clear(d): true, ontable(c): true, ontable(a): true, ontable(b): true, ontable(d): true, handempty: true, on(d, d): false, on(b, d): false, on(a, d): false, on(c, d): false, on(d, b): false, on(b, b): false, on(a, b): false, on(c, b): false, on(d, a): false, on(b, a): false, on(a, a): false, on(c, a): false, on(d, c): false, on(b, c): false, on(a, c): false, on(c, c): false, holding(d): false, holding(b): false, holding(a): false, holding(c): false}
2024-05-31 10:51:35,899 | skdecide.utils | DEBUG | Action: pick-up(b)


NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_environment().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 65 of `/Users/teichteil_fl/Projects/SkDecide/skdecide-icaps24-tutorial/.env/lib/python3.10/site-packages/skdecide/hub/solver/up/up.py`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.
INFO     translator stdin: None
INFO     translator time limit: None
INFO     translator memory limit: None
INFO     translator command line string: /Users/teichteil_fl/Projects/SkDecide/skdecide-icaps24-tutorial/.env/bin/python /Users/teichteil_fl/Projects/SkDecide/skdecide-icaps24-tutorial

2024-05-31 10:51:35,936 | skdecide.utils | DEBUG | Action: stack(b, a)
2024-05-31 10:51:35,971 | skdecide.utils | DEBUG | Action: pick-up(c)
2024-05-31 10:51:36,009 | skdecide.utils | DEBUG | Action: stack(c, b)
2024-05-31 10:51:36,045 | skdecide.utils | DEBUG | Action: pick-up(d)
2024-05-31 10:51:36,083 | skdecide.utils | DEBUG | Action: stack(d, c)
2024-05-31 10:51:36,084 | skdecide.utils | DEBUG | Episode 1 terminated after 7 steps.
2024-05-31 10:51:36,084 | skdecide.utils | INFO | The goal was reached in episode 1.


However, thanks to the unified API of scikit-decide, we can also call scikit-decide's native planners - which do not need to be specifically designed for PDDL problems! - which are compatible with the features of `UPDomain`.

Looking more closely to `UPDomain`'s characteristics, we see that it inherits from `DeterministicPlanningDomain`, which is itself a shortcut for the following features: `Domain`, `SingleAgent`, `Sequential`, `DeterministicTransitions`, `Actions`, `Goals`, `DeterministicInitialized`, `Markovian`, `FullyObservable`, and `PositiveCosts`.

Especially, scikit-decide's implementation of the [Iterated Width](https://dl.acm.org/doi/10.5555/3007337.3007433) planner is compatible with such characteristics. In order to be able to computey Iterated Width's novelty measures, we must provide the state features as vectors. In order to do so, we pass the parameter `state_encoding='vector'` to the `UPDomain` instance's constructor. The state feature vector used by Iterated Width will then just be the state vector itself.

In [5]:
domain_factory = lambda: UPDomain(up_problem, state_encoding='vector')
domain = domain_factory()

with IW(domain_factory=domain_factory,
        state_features=lambda d, s: s,
        node_ordering=lambda a_gscore, a_novelty, a_depth, b_gscore, b_novelty, b_depth: a_novelty > b_novelty
) as solver:
    solver.solve()
    rollout(
        domain,
        solver,
        num_episodes=1,
        max_steps=100,
        max_framerate=30,
        outcome_formatter=None,
    )

2024-05-31 10:51:38,277 | skdecide.utils | DEBUG | Logger is in verbose mode: all debug messages will be there for you to enjoy （〜^∇^ )〜
2024-05-31 10:51:38,278 | skdecide.utils | DEBUG | Episode 1 started with following observation:
2024-05-31 10:51:38,278 | skdecide.utils | DEBUG | [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-05-31 10:51:38,951 | skdecide.utils | DEBUG | Action: action pick-up_b {
    preconditions = [
      clear(b)
      ontable(b)
      handempty
    ]
    effects = [
      ontable(b) := false
      clear(b) := false
      handempty := false
      holding(b) := true
    ]
  }


[2024-05-31 10:51:37.587] [info] Running sequential IW solver from state [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2024-05-31 10:51:37.587] [info] Running sequential IW(1) solver from state [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2024-05-31 10:51:37.649] [info] IW(1) could not find a solution from state [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2024-05-31 10:51:37.649] [info] Running sequential IW(2) solver from state [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2024-05-31 10:51:37.813] [info] IW(2) could not find a solution from state [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2024-05-31 10:51:37.814] [info] Running sequential IW(3) solver from state [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2024-05-31 10:51:38.270] [info] Found a goal state: [0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0] (cost=6; best=6)
[2024-05-31 10:51:38.270] [info] IW(3) finished to solve 

2024-05-31 10:51:38,953 | skdecide.utils | DEBUG | Action: action stack_b_a {
    preconditions = [
      holding(b)
      clear(a)
    ]
    effects = [
      holding(b) := false
      clear(a) := false
      clear(b) := true
      handempty := true
      on(b, a) := true
    ]
  }
2024-05-31 10:51:38,991 | skdecide.utils | DEBUG | Action: action pick-up_c {
    preconditions = [
      clear(c)
      ontable(c)
      handempty
    ]
    effects = [
      ontable(c) := false
      clear(c) := false
      handempty := false
      holding(c) := true
    ]
  }
2024-05-31 10:51:39,029 | skdecide.utils | DEBUG | Action: action stack_c_b {
    preconditions = [
      holding(c)
      clear(b)
    ]
    effects = [
      holding(c) := false
      clear(b) := false
      clear(c) := true
      handempty := true
      on(c, b) := true
    ]
  }
2024-05-31 10:51:39,067 | skdecide.utils | DEBUG | Action: action pick-up_d {
    preconditions = [
      clear(d)
      ontable(d)
      handempty
    

## Using scikit-decide solvers from Unified Planning

Scikit-decide provides a Unified Planning engine which converts a Unified Planning domain into a `skdecide.hub.domain.UPDomain`, then forward the solving process to a compatible scikit-decide's solver.

First, we must retrieve the `up-skdecide` [code](https://github.com/aiplan4eu/up-skdecide) from AIPlan4EU's GitHub project.

In [ ]:
!pip install git+https://github.com/aiplan4eu/up-skdecide.git

In the following, we define a robot moving problem with *simulated action effects* which are typically hard to be handled by PDDL solvers. Scikit-decide solvers like Reinforcement Learning ones or Iterated Width are not specific to PDDL logics, and are thus generally (much) less efficient than PDDL-specific solvers, but they can naturally handle simulated action effects.

In the example below, we simulate the battery discharge of the robot when it is moving, which is usually the result of complex underlying physics simulation that cannot be easily modeled in basic PDDL in real problems.

In [21]:
Location = UserType("Location")
robot_at = up.model.Fluent("robot_at", BoolType(), l=Location)
battery_charge = Fluent('battery_charge', IntType(0, 100))
connected = up.model.Fluent(
    "connected", BoolType(), l_from=Location, l_to=Location
)

move = up.model.InstantaneousAction(
    "move", l_from=Location, l_to=Location
)
l_from = move.parameter("l_from")
l_to = move.parameter("l_to")
move.add_precondition(connected(l_from, l_to))
move.add_precondition(robot_at(l_from))
move.add_precondition(GE(battery_charge(), 10))
move.add_effect(robot_at(l_from), False)
move.add_effect(robot_at(l_to), True)
def fun(problem, state, actual_params):
    value = state.get_value(battery_charge()).constant_value()
    return [Int(value - 10)]
move.set_simulated_effect(SimulatedEffect([battery_charge()], fun))

problem = up.model.Problem("robot")
problem.add_fluent(robot_at, default_initial_value=False)
problem.add_fluent(connected, default_initial_value=False)
problem.add_action(move)

NLOC = 10
locations = [up.model.Object("l%s" % i, Location) for i in range(NLOC)]
problem.add_objects(locations)

problem.set_initial_value(robot_at(locations[0]), True)
for i in range(NLOC - 1):
    problem.set_initial_value(connected(locations[i], locations[i + 1]), True)
problem.set_initial_value(battery_charge(), 100)

problem.add_goal(robot_at(locations[-1]))

problem.add_quality_metric(
    up.model.metrics.MinimizeActionCosts({move: 1})
)

Now we call scikit-decide's implementation of Iterated Width on this problem, using Unified Planning's engine calling process and standards. We pass the parameters to be given to `skdecide.hub.solver.IW`, especially the state encoding required to compute the novelty measure, in the `config` field of the `params` dictionary of the `OneshotPlanner`.

In [22]:
get_environment().factory.add_engine("skdecide", "up_skdecide.engine", "EngineImpl")

with OneshotPlanner(problem_kind=problem.kind,
                    name='skdecide',
                    params={
                        "solver": IW,
                        "config": {"state_encoding": 'vector', "state_features": lambda d, s: s},
                    },) as planner:
    result = planner.solve(problem)
    print("%s returned: %s" % (planner.name, result.plan))

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 3 of `/var/folders/nh/hzyt86t51fxcj7rdyzbqhf280000gn/T/ipykernel_42964/3575256087.py`, you are using the following planning engine:
  * Engine name: Scikit-decide
  * Developers:  Airbus AI Research
  * Description: Scikit-decide is an AI framework for Reinforcement Learning, Automated Planning and Scheduling.

[2024-05-31 11:02:46.856] [info] Running sequential IW solver from state [  1 100   0   0   0   0   0   0   0   0   0]
[2024-05-31 11:02:46.856] [info] Running sequential IW(1) solver from state [  1 100   0   0   0   0   0   0   0   0   0]
SkDecide returned: SequentialPlan:
    move_l0_l1
    move_l1_l2
    move_l2_l3
    move_l3_l4
    move_l4_l5
    move_l5_l6
    move_l6_l7
    move_l7_l8
    move_l8_l9
[2024-05-31 11:02:46.904] [info] Found a goal state: [ 0 10  0  0  0  0  0  0  0  0  1] (cost=9; best=9)
[2024-05-31 11:02:46.904] [info] IW(1) finished to solve from state [  1 100   0   0   0   0   0   0   0  

/Users/teichteil_fl/Projects/SkDecide/skdecide-icaps24-tutorial/.env/lib/python3.10/site-packages/unified_planning/engines/mixins/oneshot_planner.py:76: UserWarning: We cannot establish whether SkDecide can solve this problem!
  warn(msg)


We show below that solving the same Unified Planning problem with RLlib's DQN algorithm comes to just change one line of code.

<div class="alert alert-block alert-info"><b>Note: </b> Scikit-decide's implementation of `skdecide.hub.solver.RayRLlib` automatically manages action filtering in the deep value and policy networks passed to the underlying RLlib's solver. It means that Unified Planning (PDDL) action preconditions are processed in the background by scikit-decide to automatically provide filtered actions to RLlib's deep networks, which is usually much more efficient than filtering those actions by means of high penalty costs on the infeasible actions. This automatic action filtering is currently only feasible with skdecide.hub.solver.ray_rllib.RayRLlib, not yet with skdecide.hub.solver.stable_baselines.StableBaseline. </div>

In [23]:
with OneshotPlanner(
    problem_kind=problem.kind,
    name="skdecide",
    params={
        "solver": RayRLlib,
        "config": {"state_encoding": "vector", "action_encoding": "int", "algo_class": DQN, "train_iterations": 1},
    },
) as planner:
    result = planner.solve(problem)
    print("%s returned: %s" % (planner.name, result.plan))

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/var/folders/nh/hzyt86t51fxcj7rdyzbqhf280000gn/T/ipykernel_42964/2555211931.py`, you are using the following planning engine:
  * Engine name: Scikit-decide
  * Developers:  Airbus AI Research
  * Description: Scikit-decide is an AI framework for Reinforcement Learning, Automated Planning and Scheduling.



2024-05-31 11:03:06,177	INFO worker.py:1749 -- Started a local Ray instance.
/Users/teichteil_fl/Projects/SkDecide/skdecide-icaps24-tutorial/.env/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:521: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/teichteil_fl/Projects/SkDecide/skdecide-icaps24-tutorial/.env/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logd

SkDecide returned: SequentialPlan:
    move_l0_l1
    move_l1_l2
    move_l2_l3
    move_l3_l4
    move_l4_l5
    move_l5_l6
    move_l6_l7
    move_l7_l8
    move_l8_l9


## Flight planning

In [ ]:
!pip install folium

In [18]:


from math import sqrt, cos, sin , pi

In [19]:
class H_Action(Enum):
    """
    Horizontal action that can be perform by the aircraft
    """

    up = -1
    straight = 0
    down = 1


class V_Action(Enum):
    """
    Vertical action that can be perform by the aircraft
    """

    climb = 1
    cruise = 0
    descent = -1

def plot_map(path, G, domain):
    m = folium.Map(
        location=[
            0.5 * (domain.lat1 + domain.lat2),
            0.5 * (domain.lon1 + domain.lon2)
        ],
        zoom_start=5)
    
    for f in G.nodes:
        folium.Marker(
            location=[
                domain.network[f[0]][f[1]][f[2]].lat,
                domain.network[f[0]][f[1]][f[2]].lon
            ],
            popup=str(f),
            icon=folium.Icon(color="beige"),
        ).add_to(m)

    for (f,t) in G.edges:
        folium.PolyLine(
            locations=[
                (domain.network[f[0]][f[1]][f[2]].lat, domain.network[f[0]][f[1]][f[2]].lon),
                (domain.network[t[0]][t[1]][t[2]].lat, domain.network[t[0]][t[1]][t[2]].lon)
            ],
            color='beige'
        ).add_to(m)

    folium.Marker(
        location=[
            domain.lat1,
            domain.lon1
        ],
        popup="origin",
        icon=folium.Icon(color="blue"),
    ).add_to(m)
    folium.Marker(
        location=[
            domain.lat2,
            domain.lon2
        ],
        popup="arrival",
        icon=folium.Icon(color="red"),
    ).add_to(m)

    for i in range(len(path)-1):
        p=path[i]
        pp=path[i+1]
        folium.Marker(
            location=[
                domain.network[p[0]][p[1]][p[2]].lat,
                domain.network[p[0]][p[1]][p[2]].lon
            ],
            popup=str(p),
            icon=folium.Icon(color="green"),
        ).add_to(m)
        folium.PolyLine(
            locations=[
                ( domain.network[p[0]][p[1]][p[2]].lat, domain.network[p[0]][p[1]][p[2]].lon),
                (domain.network[pp[0]][pp[1]][pp[2]].lat, domain.network[pp[0]][pp[1]][pp[2]].lon)
            ],
            color='green'
        ).add_to(m)
    return m

def cost(domain,f,t):
    EARTH_RADIUS = 3440 
    wp1 = domain.network[f[0]][f[1]][f[2]]
    wp2 = domain.network[t[0]][t[1]][t[2]]

    AIRCRAFT_SPEED = 500
    WIND_DIRECTION = domain.weather_interpolator.interpol_wind_classic(wp1.lat,wp1.lon,0,34*wp1.height)[1] * 180
    WIND_SPEED = domain.weather_interpolator.interpol_wind_classic(wp1.lat,wp1.lon,0,34*wp1.height)[0]
    # Computes coordinates of the direction vector in the Earth-centered system
    dir_x = EARTH_RADIUS * (
        (cos(wp2.lat * pi / 180.0) * cos(wp2.lon * pi / 180.0))
        - (cos(wp1.lat * pi / 180.0) * cos(wp1.lon * pi / 180.0))
    )
    dir_y = EARTH_RADIUS * (
        (cos(wp2.lat * pi / 180.0) * sin(wp2.lon * pi / 180.0))
        - (cos(wp1.lat * pi / 180.0) * sin(wp1.lon * pi / 180.0))
    )
    dir_z = EARTH_RADIUS * (
        sin(wp2.lat * pi / 180.0) - sin(wp1.lat * pi / 180.0)
    )
    # Computes coordinates of the direction vector in the tangential plane at the waypoint node.data
    dir_a = (-dir_x * sin(wp1.lon * pi / 180.0)) + (
        dir_y * cos(wp1.lon * pi / 180.0)
    )
    dir_b = (
        (
            dir_x
            * (
                -sin(wp1.lat * pi / 180.0)
                * cos(wp1.lon * pi / 180.0)
            )
        )
        + (
            dir_y
            * (
                -sin(wp1.lat * pi / 180.0)
                * sin(wp1.lon * pi / 180.0)
            )
        )
        + (dir_z * cos(wp1.lat * pi / 180.0))
    )
    # Normalize the direction vector
    dir_na = dir_a / sqrt(dir_a * dir_a + dir_b * dir_b)
    dir_nb = dir_b / sqrt(dir_a * dir_a + dir_b * dir_b)
    # Compute wind vector in the tangential plane
    w_a = WIND_SPEED * sin(WIND_DIRECTION * pi / 180.0)
    w_b = WIND_SPEED * cos(WIND_DIRECTION * pi / 180.0)
    # Compute speed along direction vector
    mu = (dir_na * w_a) + (dir_nb * w_b)
    phi = (
        (mu * mu)
        - (WIND_SPEED * WIND_SPEED)
        + (AIRCRAFT_SPEED * AIRCRAFT_SPEED)
    )
    dir_speed = mu + sqrt(phi)
    flown_distance = (
        AIRCRAFT_SPEED * sqrt(dir_a * dir_a + dir_b * dir_b) / dir_speed
    )
    return flown_distance

print('Generating problems')

problem = Problem("flight_planning")

#Objects
waypoint = UserType("waypoint")

#Fluents
Cost = Fluent('COST', RealType(), l_from=waypoint, l_to= waypoint)
Connected = Fluent('CONNECTED', BoolType(), l_from=waypoint, l_to= waypoint)
at = Fluent('at', BoolType(), w=waypoint)
Fuel = Fluent('Fuel', RealType())

problem.add_fluent(Cost,default_initial_value=1000000)
problem.add_fluent(Connected,default_initial_value=False)
problem.add_fluent(at,default_initial_value=False)
problem.add_fluent(Fuel)

#Actions
GoTo = InstantaneousAction("goto", fromwp = waypoint, towp=waypoint)
fromwp = GoTo.parameter('fromwp')
towp = GoTo.parameter('towp') 
#GoTo.add_precondition(GE(Fuel,Cost(fromwp,towp)))
GoTo.add_precondition(Connected(fromwp,towp))
GoTo.add_precondition(at(fromwp))
GoTo.add_effect(at(towp),True)
GoTo.add_effect(at(fromwp),False)
#GoTo.add_effect(Fuel, Minus(Fuel,Cost(fromwp,towp)))

problem.add_quality_metric(
        up.model.metrics.MinimizeActionCosts({GoTo:Cost(fromwp,towp)})
        )

problem.add_action(GoTo)

origin = "LFPG"
destination = "LFLL"
aircraft = "A320"
today = datetime.date.today()
month = 1 // 4 * 4 + 1  # will result in january, may, or september
year = today.year
day = 1
weather_date = WeatherDate(day=day, month=month, year=year)
heuristic = "lazy_fuel"
cost_function = "fuel"

domain_factory = lambda: FlightPlanningDomain(
    origin,
    destination,
    aircraft,
    weather_date=weather_date,
    heuristic_name=heuristic,
    objective=cost_function,
    fuel_loop=False,
    graph_width="large",
)
domain = domain_factory()

L = list()
x=0
y=0
z=0
for x1 in domain.network:  # len(domain.network)
    for x2 in x1:   # len(domain.network[x1])
        for x3 in range(1):
            L.append((x,y,z))
            z+=1
        z=0
        y+=1
    y=0
    x+=1

G = nx.Graph()

for point in L:
    G.add_node(point)

for i in range(len(L)):
    x1, y1, z = L[i]
    for j in range(len(L)):
        x2, y2, z = L[j]
        if (x2 == x1 + 1) and ( (y2 == y1-1) or (y2 == y1) or (y2 == y1+1) ):
            G.add_edge(L[i], L[j])

locations = {str(l) : Object(str(l),waypoint) for l in G.nodes}
problem.add_objects(locations.values())

DEST = str((40, 10, 0))
problem.set_initial_value(at(locations[str((0,0,0))]),True)
problem.set_initial_value(at(locations[str((0,1,0))]),True)
problem.set_initial_value(at(locations[str((0,2,0))]),True)
problem.set_initial_value(at(locations[str((0,3,0))]),True)
problem.set_initial_value(at(locations[str((0,4,0))]),True)
problem.set_initial_value(at(locations[str((0,5,0))]),True)
problem.set_initial_value(at(locations[str((0,6,0))]),True)
problem.set_initial_value(at(locations[str((0,7,0))]),True)
problem.set_initial_value(at(locations[str((0,8,0))]),True)
problem.set_initial_value(at(locations[str((0,9,0))]),True)
problem.set_initial_value(at(locations[str((0,10,0))]),True)

problem.add_goal(Or(at(locations[str((40, 0, 0))]),
                    at(locations[str((40, 1, 0))]),
                    at(locations[str((40, 2, 0))]), 
                    at(locations[str((40, 3, 0))]), 
                    at(locations[str((40, 4, 0))]), 
                    at(locations[str((40, 5, 0))]), 
                    at(locations[str((40, 6, 0))]), 
                    at(locations[str((40, 7, 0))]),
                    at(locations[str((40, 8, 0))]),
                    at(locations[str((40, 9, 0))]),
                    at(locations[str((40, 10, 0))]) ))
problem.set_initial_value(Fuel,70000)

for (f,t) in G.edges:
    problem.set_initial_value(Connected(locations[str(f)],locations[str(t)]), True)
    c = cost(domain,f,t)
    problem.set_initial_value(Cost(locations[str(f)],locations[str(t)]), c)

with OneshotPlanner(problem_kind=problem.kind) as planner:
    result = planner.solve(problem)
    plan = result.plan
    if plan is not None:
        print("%s returned:" % planner.name)
        print(plan)
    else:
        print("No plan found.")

fr = list()
to = list()
actions = list()
for ai in plan.actions:
    fr.append( tuple(int(i) for i in str(ai.actual_parameters[0]).strip('(').strip(')').split(',')))
    to.append( tuple( int(i) for i in str(ai.actual_parameters[1]).strip('(').strip(')').split(',')))

for t in range(len(fr)):
    if to[t][1] == fr[t][1] -1:
        a1 = H_Action.down
    if to[t][1] == fr[t][1]:
        a1 = H_Action.straight
    if to[t][1] == fr[t][1] +1:
        a1 = H_Action.up

    if to[t][2] == fr[t][2] -1:
        a2 = V_Action.descent
    if to[t][2] == fr[t][2]:
        a2 = V_Action.cruise
    if to[t][2] == fr[t][2] +1:
        a2 = V_Action.climb
    
    actions.append((a1, a2))

plot_map(to, G, domain)

Generating problems
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 262 of `/var/folders/nh/hzyt86t51fxcj7rdyzbqhf280000gn/T/ipykernel_42964/1068376017.py`, you are using the following planning engine:
  * Engine name: ENHSP
  * Developers:  Enrico Scala
  * Description: Expressive Numeric Heuristic Search Planner.



/Users/teichteil_fl/Projects/SkDecide/skdecide-icaps24-tutorial/.env/lib/python3.10/site-packages/unified_planning/io/pddl_writer.py:207: UserWarning: The PDDL printer cannot exactly represent the real constant '6441131606545367/1125899906842624'
  warn(
/Users/teichteil_fl/Projects/SkDecide/skdecide-icaps24-tutorial/.env/lib/python3.10/site-packages/unified_planning/io/pddl_writer.py:207: UserWarning: The PDDL printer cannot exactly represent the real constant '6329790793310163/1125899906842624'
  warn(
/Users/teichteil_fl/Projects/SkDecide/skdecide-icaps24-tutorial/.env/lib/python3.10/site-packages/unified_planning/io/pddl_writer.py:207: UserWarning: The PDDL printer cannot exactly represent the real constant '3121428046640097/562949953421312'
  warn(
/Users/teichteil_fl/Projects/SkDecide/skdecide-icaps24-tutorial/.env/lib/python3.10/site-packages/unified_planning/io/pddl_writer.py:207: UserWarning: The PDDL printer cannot exactly represent the real constant '6181388187945523/1125899

enhsp returned:
SequentialPlan:
    goto((0, 5, 0), (1, 5, 0))
    goto((1, 5, 0), (2, 5, 0))
    goto((2, 5, 0), (3, 5, 0))
    goto((3, 5, 0), (4, 5, 0))
    goto((4, 5, 0), (5, 5, 0))
    goto((5, 5, 0), (6, 5, 0))
    goto((6, 5, 0), (7, 5, 0))
    goto((7, 5, 0), (8, 5, 0))
    goto((8, 5, 0), (9, 5, 0))
    goto((9, 5, 0), (10, 5, 0))
    goto((10, 5, 0), (11, 5, 0))
    goto((11, 5, 0), (12, 5, 0))
    goto((12, 5, 0), (13, 5, 0))
    goto((13, 5, 0), (14, 5, 0))
    goto((14, 5, 0), (15, 5, 0))
    goto((15, 5, 0), (16, 6, 0))
    goto((16, 6, 0), (17, 5, 0))
    goto((17, 5, 0), (18, 5, 0))
    goto((18, 5, 0), (19, 5, 0))
    goto((19, 5, 0), (20, 5, 0))
    goto((20, 5, 0), (21, 5, 0))
    goto((21, 5, 0), (22, 5, 0))
    goto((22, 5, 0), (23, 5, 0))
    goto((23, 5, 0), (24, 5, 0))
    goto((24, 5, 0), (25, 5, 0))
    goto((25, 5, 0), (26, 5, 0))
    goto((26, 5, 0), (27, 5, 0))
    goto((27, 5, 0), (28, 5, 0))
    goto((28, 5, 0), (29, 5, 0))
    goto((29, 5, 0), (30, 5, 0